This notebook was written by Richie to build a lightGBM model predicting the number of calls a subscriber will make. Why do some subscribers make more calls that others? We aim to find out!

In [ ]:
import os
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
user_data = pd.read_parquet('Viamo_data.parquet')
user_data.head()

In [ ]:
user_data3.dropna(subset = ['n_calls'],inplace=TRUE)

In [ ]:
import lightgbm as ltb
from sklearn import metrics
Richie_seed = 14

chosen_columns = ['gender_numeric','age_numeric','median_call_duration','n_topics','n_themes', 'fav_theme_numeric', 'second_fav_theme_numeric','fav_topic_numeric', 'second_fav_topic_numeric']

X = user_data3.loc[:,chosen_columns].astype(int)
y = user_data3.loc[:,'n_calls'].astype(int)

In [ ]:
lgbm_mod = ltb.LGBMRegressor(n_estimators = 1000, random_state=Richie_seed)
lgbm_mod.fit(X,y)

In [ ]:
y_pred = lgbm_mod.predict(X)
y_pred[y_pred<1] = 1
print(metrics.r2_score(y, y_pred))
print(metrics.mean_squared_log_error(y , y_pred))

In [ ]:
import shap
lgbm_explainer = shap.TreeExplainer(lgbm_mod)

X_sample = X.sample(50000)

shap_values = lgbm_explainer.shap_values(X_sample)

In [ ]:
shap.summary_plot(
    shap_values = shap_values,
    features = X_sample,
    plot_type = 'dot',
    plot_size = [20,10]
)

In [ ]:
shap.dependence_plot('age_numeric', shap_values, X_sample)